# Reading Input Files

In [57]:
import pandas as pd

m = pd.read_csv('D:\\Users\\Raj Jagtap\\Documents\\AI_Lab_Assignments\\Assignment_2\\movies.csv')
r = pd.read_csv('D:\\Users\\Raj Jagtap\\Documents\\AI_Lab_Assignments\\Assignment_2\\ratings.csv')

Movies CSV

In [80]:
m.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Rating CSV

In [81]:
r.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


 Grouping rating data and finding mean rating

In [82]:
ratings = pd.DataFrame(r.groupby('movieId')['rating'].mean())

Merging average rating with movie titles and genres

In [83]:
ratings = pd.merge(ratings, m, on='movieId')

New Ratings DataFrame

In [84]:
ratings.head()

,movieId,rating,title,genres
0,1,3.886649,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,3.246583,Jumanji (1995),Adventure|Children|Fantasy
2,3,3.173981,Grumpier Old Men (1995),Comedy|Romance
3,4,2.874540,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,3.077291,Father of the Bride Part II (1995),Comedy


Dropping Null value rows

In [85]:
ratings.dropna(inplace = True) 

Splitting genres string to arrays

In [86]:
ratings["genres"]= ratings["genres"].str.split("|")

Updated Ratings DataFrame

In [87]:
ratings.head()

,movieId,rating,title,genres
0,1,3.886649,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,3.246583,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,3.173981,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,2.874540,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,3.077291,Father of the Bride Part II (1995),[Comedy]


Finding all genres

In [88]:
genres = []
for i in ratings['genres']:
    for j in i:
        if j not in genres:
            genres.append(j)
#             print(j)

In [89]:
print(genres)

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical', 'Western', 'Film-Noir', '(no genres listed)']


Making a binary array for genres

In [90]:
genres_coded = []
for i in ratings['genres']:
    gen = []
    for k in genres:
        if k in i:
            gen.append(1)
            
        else:
            gen.append(0)
            
#     print(gen)
    genres_coded.append(gen)
    
ratings['genres_coded'] = pd.Series(genres_coded, index = ratings.index)
# print(ratings['genres_coded'])

Making model using sklearn

In [91]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

Training model

In [92]:
model.fit(genres_coded)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

Defining a function for computing array of recommended movies using KNN for genres and ratings

In [93]:
import numpy as np
def recommend_similar(ratings, movieId, model, n_neighbors=10):
    code = []
    ratingOfMovie = []
    for i in range(len(ratings)):
        if ratings['movieId'][i] == movieId:
            code.append(ratings['genres_coded'][i])
            ratingOfMovie.append(ratings['rating'][i])
            print('Movie Entered :', ratings['title'][i], ratings['genres'][i], ratings['rating'][i])
#     print(code)
#     print(ratingOfMovie)
    reco = model.kneighbors(code, n_neighbors=n_neighbors, return_distance=True)
#     print(reco)
    recommender_queue = []
    for i in range(len(reco[0][0])):
#         print(np.absolute(ratings['rating'][reco[1][0][i]]-ratingOfMovie[0]))
        recommender_queue.append([ratings['genres'][reco[1][0][i]], reco[0][0][i]+np.absolute(ratings['rating'][reco[1][0][i]]-ratingOfMovie[0])/5, reco[1][0][i], ratings['title'][reco[1][0][i]], ratings['rating'][reco[1][0][i]]])
    
    recommender_queue = sorted(recommender_queue)
#     recommeded = []
    for i in recommender_queue:
        print([i[2], i[3], i[0], i[4]])
#     print(recommender_queue)


In [94]:
movieId = input("Enter Movie Id : ")
movieId = int(movieId)
recommend_similar(ratings, movieId, model)

Enter Movie Id : 12
Movie Entered : Dracula: Dead and Loving It (1995) ['Comedy', 'Horror'] 2.6696507515473034
[20236, 'I Bought a Vampire Motorcycle (1990)', ['Comedy', 'Horror'], 2.6666666666666665]
[29458, 'Night of the Living Dorks (2004)', ['Comedy', 'Horror'], 2.625]
[13592, 'Stacy (2001)', ['Comedy', 'Horror'], 2.7941176470588234]
[20240, 'Ice Cream Man (1995)', ['Comedy', 'Horror'], 2.5357142857142856]
[5109, 'Motel Hell (1980)', ['Comedy', 'Horror'], 2.9148471615720526]
[10991, 'Scary Movie 4 (2006)', ['Comedy', 'Horror'], 2.3270864567716143]
[6396, 'Ed and His Dead Mother (1993)', ['Comedy', 'Horror'], 3.056338028169014]
[5107, 'The Monster Club (1981)', ['Comedy', 'Horror'], 3.0714285714285716]
[45059, 'Popcorn (2007)', ['Comedy', 'Horror'], 2.0]
[34149, 'Ghost Writer (2007)', ['Comedy', 'Horror'], 3.9545454545454546]
